In [39]:
!pip install kaggle

In [40]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# Move the kaggle.json file to the correct directory
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [41]:
!kaggle datasets download -d phylake1337/fire-dataset

fire-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [42]:
!unzip -q fire-dataset.zip

replace fire_dataset/fire_images/fire.1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [79]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [82]:
# Set your data path
data_path = '/content/fire_dataset'

In [83]:
# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [84]:
# Create a list of all image files and labels
all_images = []
labels = []

for label in ['fire_images', 'non_fire_images']:
    label_path = os.path.join(data_path, label)
    image_files = [os.path.join(label, img) for img in os.listdir(label_path)]
    all_images.extend(image_files)
    labels.extend([label] * len(image_files))

In [85]:
# Split the data into training, validation, and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(all_images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [86]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [87]:
# No data augmentation for validation/test images
test_datagen = ImageDataGenerator(rescale=1./255)

In [88]:
# Load training data
train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'filename': train_images, 'label': train_labels}),
    directory=data_path,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 639 validated image filenames belonging to 2 classes.


In [89]:
# Load validation data
val_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'filename': val_images, 'label': val_labels}),
    directory=data_path,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 160 validated image filenames belonging to 2 classes.


In [90]:
# Load test data
test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'filename': test_images, 'label': test_labels}),
    directory=data_path,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 200 validated image filenames belonging to 2 classes.
